<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/265_MissionOrchestratorAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KPI tracking utilities for Mission Orchestrator Agent

In [ ]:
"""KPI tracking utilities for Mission Orchestrator Agent"""

from typing import List, Dict, Any


def calculate_kpi_metrics(
    executed_tasks: List[Dict[str, Any]],
    mission_kpis: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Calculate current KPI metrics based on executed tasks.

    Args:
        executed_tasks: List of completed task execution results
        mission_kpis: Mission KPI definitions

    Returns:
        Dictionary of calculated KPI metrics
    """
    metrics = {}

    # Calculate total time (sum of all task durations)
    total_duration_minutes = sum(
        task.get("duration_minutes", 0) for task in executed_tasks
        if task.get("status") == "completed"
    )

    # Convert to days (for onboarding time KPI)
    total_duration_days = total_duration_minutes / (60 * 24)

    # Calculate number of steps (completed tasks)
    steps_completed = len([
        task for task in executed_tasks
        if task.get("status") == "completed"
    ])

    # Map to mission-specific KPIs
    if "target_onboarding_time_days" in mission_kpis:
        metrics["actual_onboarding_time_days"] = round(total_duration_days, 2)

    if "target_pipeline_days" in mission_kpis:
        metrics["actual_pipeline_days"] = round(total_duration_days, 2)

    if "target_resolution_time_hours" in mission_kpis:
        metrics["actual_resolution_time_hours"] = round(total_duration_minutes / 60.0, 2)

    if "max_steps" in mission_kpis:
        metrics["actual_steps"] = steps_completed

    if "min_touchpoints" in mission_kpis:
        metrics["actual_touchpoints"] = steps_completed

    # Calculate improvement percentage if baseline exists
    if "baseline_onboarding_time_days" in mission_kpis:
        baseline = mission_kpis["baseline_onboarding_time_days"]
        actual = metrics.get("actual_onboarding_time_days", 0)
        if baseline > 0:
            improvement = ((baseline - actual) / baseline) * 100
            metrics["improvement_percentage"] = round(improvement, 2)

    if "baseline_pipeline_days" in mission_kpis:
        baseline = mission_kpis["baseline_pipeline_days"]
        actual = metrics.get("actual_pipeline_days", 0)
        if baseline > 0:
            improvement = ((baseline - actual) / baseline) * 100
            metrics["improvement_percentage"] = round(improvement, 2)

    if "baseline_resolution_time_hours" in mission_kpis:
        baseline = mission_kpis["baseline_resolution_time_hours"]
        actual = metrics.get("actual_resolution_time_hours", 0)
        if baseline > 0:
            improvement = ((baseline - actual) / baseline) * 100
            metrics["improvement_percentage"] = round(improvement, 2)

    return metrics


def assess_kpi_status(
    kpi_metrics: Dict[str, Any],
    mission_kpis: Dict[str, Any],
    warning_threshold: float = 0.8,
    critical_threshold: float = 0.5
) -> Dict[str, str]:
    """
    Assess KPI status (on_track, at_risk, exceeded).

    Args:
        kpi_metrics: Calculated KPI metrics
        mission_kpis: Mission KPI definitions with targets
        warning_threshold: Warning threshold (0.8 = 80% of target)
        critical_threshold: Critical threshold (0.5 = 50% of target)

    Returns:
        Dictionary mapping KPI names to status strings
    """
    status = {}

    # Check onboarding time KPI
    if "target_onboarding_time_days" in mission_kpis and "actual_onboarding_time_days" in kpi_metrics:
        target = mission_kpis["target_onboarding_time_days"]
        actual = kpi_metrics["actual_onboarding_time_days"]

        if actual <= target:
            status["onboarding_time"] = "exceeded"  # Better than target
        elif actual <= target / critical_threshold:
            status["onboarding_time"] = "on_track"
        elif actual <= target / warning_threshold:
            status["onboarding_time"] = "at_risk"
        else:
            status["onboarding_time"] = "at_risk"

    # Check pipeline days KPI
    if "target_pipeline_days" in mission_kpis and "actual_pipeline_days" in kpi_metrics:
        target = mission_kpis["target_pipeline_days"]
        actual = kpi_metrics["actual_pipeline_days"]

        if actual <= target:
            status["pipeline_days"] = "exceeded"
        elif actual <= target / critical_threshold:
            status["pipeline_days"] = "on_track"
        elif actual <= target / warning_threshold:
            status["pipeline_days"] = "at_risk"
        else:
            status["pipeline_days"] = "at_risk"

    # Check resolution time KPI
    if "target_resolution_time_hours" in mission_kpis and "actual_resolution_time_hours" in kpi_metrics:
        target = mission_kpis["target_resolution_time_hours"]
        actual = kpi_metrics["actual_resolution_time_hours"]

        if actual <= target:
            status["resolution_time"] = "exceeded"
        elif actual <= target / critical_threshold:
            status["resolution_time"] = "on_track"
        elif actual <= target / warning_threshold:
            status["resolution_time"] = "at_risk"
        else:
            status["resolution_time"] = "at_risk"

    # Check steps KPI
    if "max_steps" in mission_kpis and "actual_steps" in kpi_metrics:
        target = mission_kpis["max_steps"]
        actual = kpi_metrics["actual_steps"]

        if actual <= target:
            status["steps"] = "on_track"
        else:
            status["steps"] = "at_risk"

    # Check touchpoints KPI
    if "min_touchpoints" in mission_kpis and "actual_touchpoints" in kpi_metrics:
        target = mission_kpis["min_touchpoints"]
        actual = kpi_metrics["actual_touchpoints"]

        if actual >= target:
            status["touchpoints"] = "on_track"
        else:
            status["touchpoints"] = "at_risk"

    return status


def calculate_roi_improvement(baseline: float, actual: float) -> float:
    """
    Calculate ROI improvement percentage.

    Args:
        baseline: Baseline value
        actual: Actual value

    Returns:
        Improvement percentage (positive = improvement, negative = degradation)
    """
    if baseline == 0:
        return 0.0

    improvement = ((baseline - actual) / baseline) * 100.0
    return round(improvement, 2)



# Progress tracking utilities for Mission Orchestrator Agent

In [ ]:
"""Progress tracking utilities for Mission Orchestrator Agent"""

from typing import List, Dict, Any
from datetime import datetime


def calculate_progress(completed_count: int, total_count: int) -> float:
    """
    Calculate progress percentage.

    Args:
        completed_count: Number of completed tasks
        total_count: Total number of tasks

    Returns:
        Progress percentage (0-100)
    """
    if total_count == 0:
        return 0.0

    progress = (completed_count / total_count) * 100.0
    return round(progress, 2)


def calculate_elapsed_time(start_time: str) -> float:
    """
    Calculate elapsed time in minutes since mission start.

    Args:
        start_time: ISO format timestamp string

    Returns:
        Elapsed time in minutes
    """
    try:
        start = datetime.fromisoformat(start_time)
        now = datetime.now()
        elapsed_seconds = (now - start).total_seconds()
        return round(elapsed_seconds / 60.0, 2)
    except (ValueError, TypeError):
        return 0.0


def estimate_remaining_time(
    completed_tasks: List[Dict[str, Any]],
    remaining_tasks: List[Dict[str, Any]]
) -> float:
    """
    Estimate remaining time based on completed tasks' average duration.

    Args:
        completed_tasks: List of completed task execution results
        remaining_tasks: List of tasks not yet executed

    Returns:
        Estimated remaining time in minutes
    """
    if not completed_tasks:
        # No completed tasks yet, use estimated durations from remaining tasks
        total_estimated = sum(
            task.get("estimated_duration_minutes", 0) for task in remaining_tasks
        )
        return round(total_estimated, 2)

    # Calculate average duration from completed tasks
    completed_durations = [
        task.get("duration_minutes", 0) for task in completed_tasks
        if task.get("status") == "completed"
    ]

    if not completed_durations:
        # Fallback to estimated durations
        total_estimated = sum(
            task.get("estimated_duration_minutes", 0) for task in remaining_tasks
        )
        return round(total_estimated, 2)

    avg_duration = sum(completed_durations) / len(completed_durations)

    # Estimate remaining time based on average
    remaining_count = len(remaining_tasks)
    estimated_remaining = avg_duration * remaining_count

    return round(estimated_remaining, 2)



# Progress Tracking Node

In [ ]:
def progress_tracking_node(state: MissionOrchestratorState) -> Dict[str, Any]:
    """
    Progress Tracking Node: Calculate progress and KPI metrics.

    This node:
    1. Calculates progress percentage
    2. Calculates elapsed time
    3. Estimates remaining time
    4. Calculates KPI metrics
    5. Assesses KPI status

    Input:
        - executed_tasks (List[Dict]): Completed tasks
        - tasks_total (int): Total tasks
        - tasks_completed (int): Completed task count
        - mission_start_time (str): Mission start timestamp
        - mission_kpis (Dict): Mission KPI definitions
        - task_queue (List[Dict]): Remaining tasks

    Output:
        - progress_percentage (float): Progress 0-100
        - elapsed_time_minutes (float): Time since start
        - estimated_remaining_minutes (float): Estimated time to completion
        - kpi_metrics (Dict): Calculated KPI values
        - kpi_status (Dict): KPI achievement status
        - errors (List[str]): Any errors encountered
    """
    errors = state.get("errors", [])
    executed_tasks = state.get("executed_tasks", [])
    tasks_total = state.get("tasks_total", 0)
    tasks_completed = state.get("tasks_completed", 0)
    mission_start_time = state.get("mission_start_time")
    mission_kpis = state.get("mission_kpis", {})
    task_queue = state.get("task_queue", [])
    config = MissionOrchestratorConfig()

    try:
        # Calculate progress percentage
        progress_percentage = calculate_progress(tasks_completed, tasks_total)

        # Calculate elapsed time
        elapsed_time_minutes = 0.0
        if mission_start_time:
            elapsed_time_minutes = calculate_elapsed_time(mission_start_time)

        # Estimate remaining time
        estimated_remaining_minutes = estimate_remaining_time(executed_tasks, task_queue)

        # Calculate KPI metrics
        kpi_metrics = calculate_kpi_metrics(executed_tasks, mission_kpis)

        # Assess KPI status
        kpi_status = assess_kpi_status(
            kpi_metrics,
            mission_kpis,
            warning_threshold=config.kpi_warning_threshold,
            critical_threshold=config.kpi_critical_threshold
        )

        return {
            "progress_percentage": progress_percentage,
            "elapsed_time_minutes": elapsed_time_minutes,
            "estimated_remaining_minutes": estimated_remaining_minutes,
            "kpi_metrics": kpi_metrics,
            "kpi_status": kpi_status,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"progress_tracking_node: Unexpected error: {str(e)}"]
        }


# Standalone test script for progress tracking and KPI monitoring

In [ ]:
"""Standalone test script for progress tracking and KPI monitoring"""

from agents.mission_orchestrator.utilities.progress_tracking import (
    calculate_progress,
    calculate_elapsed_time,
    estimate_remaining_time
)
from agents.mission_orchestrator.utilities.kpi_tracking import (
    calculate_kpi_metrics,
    assess_kpi_status,
    calculate_roi_improvement
)
from agents.mission_orchestrator.nodes import progress_tracking_node
from agents.mission_orchestrator.utilities.data_loading import (
    load_mission_tasks,
    load_mission_kpis
)
from config import MissionOrchestratorState
from datetime import datetime, timedelta


def test_progress_tracking_utilities():
    """Test progress tracking utilities"""
    print("=" * 60)
    print("Testing Progress Tracking Utilities")
    print("=" * 60)

    # Test 1: Calculate progress
    print("\n1. Calculating progress...")
    progress = calculate_progress(2, 5)
    print(f"   ✓ Progress: {progress}% (2/5 tasks)")

    progress = calculate_progress(5, 5)
    print(f"   ✓ Progress: {progress}% (5/5 tasks - complete)")

    progress = calculate_progress(0, 5)
    print(f"   ✓ Progress: {progress}% (0/5 tasks - not started)")

    # Test 2: Calculate elapsed time
    print("\n2. Calculating elapsed time...")
    start_time = (datetime.now() - timedelta(minutes=15)).isoformat()
    elapsed = calculate_elapsed_time(start_time)
    print(f"   ✓ Elapsed time: {elapsed} minutes (from {start_time})")

    # Test 3: Estimate remaining time
    print("\n3. Estimating remaining time...")
    completed_tasks = [
        {"duration_minutes": 5.0, "status": "completed"},
        {"duration_minutes": 10.0, "status": "completed"}
    ]
    remaining_tasks = [
        {"estimated_duration_minutes": 8},
        {"estimated_duration_minutes": 3}
    ]

    # With completed tasks (use average)
    estimated = estimate_remaining_time(completed_tasks, remaining_tasks)
    print(f"   ✓ Estimated remaining: {estimated} minutes (based on avg of completed)")

    # Without completed tasks (use estimates)
    estimated = estimate_remaining_time([], remaining_tasks)
    print(f"   ✓ Estimated remaining: {estimated} minutes (based on task estimates)")

    print("\n" + "=" * 60)
    print("Progress Tracking Utilities Tests Complete!")
    print("=" * 60)


def test_kpi_tracking_utilities():
    """Test KPI tracking utilities"""
    print("\n" + "=" * 60)
    print("Testing KPI Tracking Utilities")
    print("=" * 60)

    # Load mission KPIs
    mission_kpis = load_mission_kpis("M001")

    # Test 1: Calculate KPI metrics
    print("\n1. Calculating KPI metrics...")
    executed_tasks = [
        {
            "task_id": "T1",
            "status": "completed",
            "duration_minutes": 5.0
        },
        {
            "task_id": "T2",
            "status": "completed",
            "duration_minutes": 10.0
        },
        {
            "task_id": "T3",
            "status": "completed",
            "duration_minutes": 3.0
        }
    ]

    kpi_metrics = calculate_kpi_metrics(executed_tasks, mission_kpis)
    print(f"   ✓ Calculated metrics:")
    for key, value in kpi_metrics.items():
        print(f"     - {key}: {value}")

    # Test 2: Assess KPI status
    print("\n2. Assessing KPI status...")
    kpi_status = assess_kpi_status(kpi_metrics, mission_kpis)
    print(f"   ✓ KPI Status:")
    for key, value in kpi_status.items():
        print(f"     - {key}: {value}")

    # Test 3: Calculate ROI improvement
    print("\n3. Calculating ROI improvement...")
    baseline = mission_kpis.get("baseline_onboarding_time_days", 5)
    actual = kpi_metrics.get("actual_onboarding_time_days", 0)
    improvement = calculate_roi_improvement(baseline, actual)
    print(f"   ✓ Improvement: {improvement}%")
    print(f"     Baseline: {baseline} days")
    print(f"     Actual: {actual} days")

    print("\n" + "=" * 60)
    print("KPI Tracking Utilities Tests Complete!")
    print("=" * 60)


def test_progress_tracking_node():
    """Test progress tracking node"""
    print("\n" + "=" * 60)
    print("Testing Progress Tracking Node")
    print("=" * 60)

    # Setup state with executed tasks
    mission_start_time = (datetime.now() - timedelta(minutes=20)).isoformat()

    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "mission_tasks": load_mission_tasks("M001"),
        "mission_kpis": load_mission_kpis("M001"),
        "executed_tasks": [
            {
                "task_id": "T1",
                "status": "completed",
                "duration_minutes": 5.0
            },
            {
                "task_id": "T2",
                "status": "completed",
                "duration_minutes": 10.0
            },
            {
                "task_id": "T3",
                "status": "completed",
                "duration_minutes": 3.0
            }
        ],
        "tasks_total": 3,
        "tasks_completed": 3,
        "task_queue": [],
        "mission_start_time": mission_start_time,
        "errors": []
    }

    result = progress_tracking_node(state)

    if "progress_percentage" in result:
        print("   ✓ Node executed successfully")
        print(f"   - Progress: {result['progress_percentage']}%")
        print(f"   - Elapsed time: {result['elapsed_time_minutes']} minutes")
        print(f"   - Estimated remaining: {result['estimated_remaining_minutes']} minutes")
        print(f"\n   KPI Metrics:")
        for key, value in result.get("kpi_metrics", {}).items():
            print(f"     - {key}: {value}")
        print(f"\n   KPI Status:")
        for key, value in result.get("kpi_status", {}).items():
            print(f"     - {key}: {value}")
    else:
        print("   ✗ Node failed")
        if result.get("errors"):
            for error in result["errors"]:
                print(f"     - {error}")

    print("\n" + "=" * 60)
    print("Progress Tracking Node Test Complete!")
    print("=" * 60)


def test_full_flow_with_progress():
    """Test full flow including progress tracking"""
    print("\n" + "=" * 60)
    print("Testing Full Flow with Progress Tracking")
    print("=" * 60)

    from agents.mission_orchestrator.nodes import (
        goal_node, planning_node, data_loading_node,
        task_ordering_node, task_execution_node
    )

    # Start with just mission_id
    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "errors": []
    }

    # Step 1: Goal node
    print("\n1. Executing goal node...")
    goal_result = goal_node(state)
    state = {**state, **goal_result}

    # Step 2: Planning node
    print("2. Executing planning node...")
    planning_result = planning_node(state)
    state = {**state, **planning_result}

    # Step 3: Data loading node
    print("3. Executing data loading node...")
    data_result = data_loading_node(state)
    state = {**state, **data_result}

    # Step 4: Task ordering node
    print("4. Executing task ordering node...")
    ordering_result = task_ordering_node(state)
    state = {**state, **ordering_result}

    # Step 5: Execute all tasks
    print("5. Executing tasks...")
    max_iterations = 10
    iteration = 0

    while state.get("task_queue") and iteration < max_iterations:
        iteration += 1
        execution_result = task_execution_node(state)
        state = {**state, **execution_result}

        # Track progress after each task
        progress_result = progress_tracking_node(state)
        state = {**state, **progress_result}

        if execution_result.get("executed_tasks"):
            last_executed = execution_result["executed_tasks"][-1]
            print(f"   Task {last_executed['task_id']} completed - Progress: {state.get('progress_percentage', 0)}%")

        if not state.get("task_queue"):
            break

    # Final progress tracking
    print("\n6. Final progress tracking...")
    progress_result = progress_tracking_node(state)
    state = {**state, **progress_result}

    print(f"\n--- Mission Execution Summary ---")
    print(f"  Mission: {state['mission']['mission_name']}")
    print(f"  Progress: {state['progress_percentage']}%")
    print(f"  Tasks: {state['tasks_completed']}/{state['tasks_total']} completed")
    print(f"  Elapsed time: {state['elapsed_time_minutes']:.2f} minutes")
    print(f"  Estimated remaining: {state['estimated_remaining_minutes']:.2f} minutes")

    print(f"\n  KPI Metrics:")
    for key, value in state.get("kpi_metrics", {}).items():
        print(f"    - {key}: {value}")

    print(f"\n  KPI Status:")
    for key, value in state.get("kpi_status", {}).items():
        status_icon = "✓" if value == "on_track" or value == "exceeded" else "⚠"
        print(f"    {status_icon} {key}: {value}")

    # Show improvement
    if "improvement_percentage" in state.get("kpi_metrics", {}):
        improvement = state["kpi_metrics"]["improvement_percentage"]
        print(f"\n  ROI Improvement: {improvement}%")
        baseline = state["mission_kpis"].get("baseline_onboarding_time_days", 0)
        actual = state["kpi_metrics"].get("actual_onboarding_time_days", 0)
        print(f"    Baseline: {baseline} days → Actual: {actual} days")

    print("\n" + "=" * 60)
    print("Full Flow with Progress Tracking Test Complete!")
    print("=" * 60)


if __name__ == "__main__":
    try:
        test_progress_tracking_utilities()
        test_kpi_tracking_utilities()
        test_progress_tracking_node()
        test_full_flow_with_progress()
        print("\n✅ All tests completed successfully!")
    except Exception as e:
        print(f"\n❌ Error during testing: {e}")
        import traceback
        traceback.print_exc()



In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_000_MissionOrchestratorAgent % python test_progress_tracking_standalone.py
============================================================
Testing Progress Tracking Utilities
============================================================

1. Calculating progress...
   ✓ Progress: 40.0% (2/5 tasks)
   ✓ Progress: 100.0% (5/5 tasks - complete)
   ✓ Progress: 0.0% (0/5 tasks - not started)

2. Calculating elapsed time...
   ✓ Elapsed time: 15.0 minutes (from 2025-12-11T16:03:57.948625)

3. Estimating remaining time...
   ✓ Estimated remaining: 15.0 minutes (based on avg of completed)
   ✓ Estimated remaining: 11 minutes (based on task estimates)

============================================================
Progress Tracking Utilities Tests Complete!
============================================================

============================================================
Testing KPI Tracking Utilities
============================================================

1. Calculating KPI metrics...
   ✓ Calculated metrics:
     - actual_onboarding_time_days: 0.01
     - actual_steps: 3
     - improvement_percentage: 99.8

2. Assessing KPI status...
   ✓ KPI Status:
     - onboarding_time: exceeded
     - steps: on_track

3. Calculating ROI improvement...
   ✓ Improvement: 99.8%
     Baseline: 5 days
     Actual: 0.01 days

============================================================
KPI Tracking Utilities Tests Complete!
============================================================

============================================================
Testing Progress Tracking Node
============================================================
   ✓ Node executed successfully
   - Progress: 100.0%
   - Elapsed time: 20.0 minutes
   - Estimated remaining: 0.0 minutes

   KPI Metrics:
     - actual_onboarding_time_days: 0.01
     - actual_steps: 3
     - improvement_percentage: 99.8

   KPI Status:
     - onboarding_time: exceeded
     - steps: on_track

============================================================
Progress Tracking Node Test Complete!
============================================================

============================================================
Testing Full Flow with Progress Tracking
============================================================

1. Executing goal node...
2. Executing planning node...
3. Executing data loading node...
4. Executing task ordering node...
5. Executing tasks...
   Task T1 completed - Progress: 33.33%
   Task T2 completed - Progress: 66.67%
   Task T3 completed - Progress: 100.0%

6. Final progress tracking...

--- Mission Execution Summary ---
  Mission: Reduce Customer Onboarding Time
  Progress: 100.0%
  Tasks: 3/3 completed
  Elapsed time: 0.01 minutes
  Estimated remaining: 0.00 minutes

  KPI Metrics:
    - actual_onboarding_time_days: 0.0
    - actual_steps: 3
    - improvement_percentage: 100.0

  KPI Status:
    ✓ onboarding_time: exceeded
    ✓ steps: on_track

  ROI Improvement: 100.0%
    Baseline: 5 days → Actual: 0.0 days

============================================================
Full Flow with Progress Tracking Test Complete!
============================================================

✅ All tests completed successfully!
